In [2]:
from keras.layers import Input, Concatenate, Dense, Embedding, Flatten, LSTM, Conv2D, MaxPool2D
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import normalize, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from imblearn.under_sampling import RandomUnderSampler
from numpy.linalg import svd

Using TensorFlow backend.


In [ ]:
d2v = np.load('/home/hzhan828/d2vecs.npy')
all_img = np.load('/home/hzhan828/kiva/all_img.npy')
new_comb = np.load('/home/hzhan828/kiva/newcomb_down.npy')
new_test_label = np.load('/home/hzhan828/kiva/new_test_label.npy')

In [ ]:
with open('/home/hzhan828/kiva/removed_indexes.pkl', 'rb') as handle:
    removed_indexes = pickle.load(handle)

In [ ]:
new_d2v = np.zeros((149750,300))
for i in range(149750):
    if i not in removed_indexes:
        new_d2v[i] = d2v[i]


In [ ]:
new_comb_norm = normalize(new_comb)
X_train, X_test, y_train, y_test, d2v_train, d2v_test,img_train, img_test = train_test_split(new_comb, new_test_label, new_d2v,all_img,test_size = 0.1, random_state = 0)


In [8]:
early_stopping_monitor = EarlyStopping(patience=2)
A1 = Input(shape=(311,),name='A1')
A2 = Dense(512, activation='relu',name='A2')(A1)
A3 = Dense(512, activation='relu',name='A3')(A2)
A4 = Dense(512, activation='relu',name='A4')(A3)
A5 = Dense(256, activation='relu',name='A5')(A4)


B1 = Input(shape=(300,),name='B1')
B2 = Dense(256, activation='relu',name='B2')(B1)
B3 = Dense(64, activation='relu',name='B3')(B2)


E1 = Input(shape=(300,300,3),name='E1')
E2 = Conv2D(5, kernel_size=3, activation='relu', 
              input_shape=(300, 300, 3),
              padding = 'valid',
              strides = 1,)(E1)
E3 = MaxPool2D(2)(E2)
E4 = Conv2D(15, kernel_size=2, activation='relu')(E3)
E5 = MaxPool2D(2)(E4)
E6 = Flatten()(E5)
E7 = Dense(160, activation='relu',name='E7')(E6)


merged = Concatenate()([A3,B3,E7])
C1 = Dense(480, activation='relu',name='C1')(merged)

C2 = Dense(160, activation='relu',name='C2')(C1)

C3 = Dense(160, activation='relu',name='C3')(C2)
C4 = Dense(1, activation='sigmoid',name='C4')(C3)
model = Model(inputs=[A1,B1,E1],outputs = C4)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
E1 (InputLayer)                 (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 298, 298, 5)  140         E1[0][0]                         
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 149, 149, 5)  0           conv2d_9[0][0]                   
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 148, 148, 15) 315         max_pooling2d_9[0][0]            
__________________________________________________________________________________________________
A1 (InputL

In [ ]:
training = model.fit([X_train,d2v_train,img_train], y_train, epochs=20, batch_size=None, validation_split=0.1, 
          verbose=True, callbacks = [early_stopping_monitor]) 

In [ ]:
model.evaluate([X_test,d2v_test,img_test], y_test)